In [1]:
from pyspark.sql import SparkSession
MAX_MEMORY = '8g'
spark = SparkSession.builder.appName("taxi-fare-prediction_2nd")\
                    .config('spark.driver.memory', MAX_MEMORY)\
                    .config('spark.excutor.memory', MAX_MEMORY)\
                    .getOrCreate()

In [2]:
import os
cwd = os.getcwd()
trip_data_path = os.path.join(cwd, 'learning_spark_data/trips','*.csv')
trip_data_path
file_path = f"file:///{trip_data_path.replace(os.sep, '/')}"
file_path

'file:////home/jovyan/work/start_spark/learning_spark_data/trips/*.csv'

In [3]:
trip_df = spark.read.csv(file_path, inferSchema=True, header=True)
trip_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [4]:
trip_df.createOrReplaceTempView('trips')

In [5]:
query = """
SELECT 
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""

In [6]:
data_df = spark.sql(query)
data_df.show(5)

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|              0|               138|                265|         16.5|          0|     Monday|       70.07|
|              1|                68|                264|         1.13|          0|     Monday|       11.16|
|              1|               239|                262|         2.68|          0|     Monday|       18.59|
|              1|               186|                 91|         12.4|          0|     Monday|        43.8|
|              2|               132|                265|          9.7|          0|     Monday|        32.3|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
only showing top 5 rows



In [7]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, StandardScaler

In [8]:
train_data, test_data = data_df.randomSplit([0.8,0.2], seed=1)

# 파이프라인 생성
- 전처리 과정을 각 스테이지로 정의해서 쌓는다
- 범주형] StringIndexer+onehotencoding 'pickup_location_id', 'dropoff_location_id', 'day_of_week'
- 수치형] StandardScaler : 'passenger_count', 'trip_distance', 'pickup_time'

In [9]:
stages = []

In [10]:
cat_features = ['pickup_location_id', 'dropoff_location_id', 'day_of_week']
for cat in cat_features:
    cat_index = StringIndexer(inputCol=cat, outputCol=cat+'_idx').setHandleInvalid('keep')
    onehot_encode = OneHotEncoder(inputCols=[cat_index.getOutputCol()], #_idx_col
                                  outputCols=[cat+'_onehot']) #postfix
    stages += [cat_index, onehot_encode] #collist
stages

[StringIndexer_8cd0e1c26018,
 OneHotEncoder_850161a4e3d5,
 StringIndexer_380fe2900eda,
 OneHotEncoder_0ec22abb137f,
 StringIndexer_8cfef571df2c,
 OneHotEncoder_700633e86b19]

In [11]:
num_features = ['passenger_count', 'trip_distance', 'pickup_time']
for num in num_features:
    num_assembler = VectorAssembler(inputCols=[num], outputCol=num+'_vector')
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol=num+'_scaled')
    stages += [num_assembler, num_scaler]
stages

[StringIndexer_8cd0e1c26018,
 OneHotEncoder_850161a4e3d5,
 StringIndexer_380fe2900eda,
 OneHotEncoder_0ec22abb137f,
 StringIndexer_8cfef571df2c,
 OneHotEncoder_700633e86b19,
 VectorAssembler_339e36fbd14d,
 StandardScaler_4a17b4b136e1,
 VectorAssembler_339c18099518,
 StandardScaler_cf4006580f92,
 VectorAssembler_89a4cafa1031,
 StandardScaler_d0373c7c34cb]

In [12]:
assembler_input = [cat+'_onehot' for cat in cat_features] + [num+'_scaled' for num in num_features]
assembler_input

['pickup_location_id_onehot',
 'dropoff_location_id_onehot',
 'day_of_week_onehot',
 'passenger_count_scaled',
 'trip_distance_scaled',
 'pickup_time_scaled']

In [13]:
assembler = VectorAssembler(
    inputCols= assembler_input,
    outputCol= 'feature_vector'
)
stages += [assembler]
stages

[StringIndexer_8cd0e1c26018,
 OneHotEncoder_850161a4e3d5,
 StringIndexer_380fe2900eda,
 OneHotEncoder_0ec22abb137f,
 StringIndexer_8cfef571df2c,
 OneHotEncoder_700633e86b19,
 VectorAssembler_339e36fbd14d,
 StandardScaler_4a17b4b136e1,
 VectorAssembler_339c18099518,
 StandardScaler_cf4006580f92,
 VectorAssembler_89a4cafa1031,
 StandardScaler_d0373c7c34cb,
 VectorAssembler_3da1c52baab8]

In [14]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)
fitted_transform = pipeline.fit(train_data)
vtrain_data = fitted_transform.transform(train_data)
vtrain_data.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vector: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vector: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vector: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (nul

In [15]:
vtrain_data.select('feature_vector').show(3)

+--------------------+
|      feature_vector|
+--------------------+
|(533,[62,311,527,...|
|(533,[62,280,526,...|
|(533,[62,280,527,...|
+--------------------+
only showing top 3 rows



In [16]:
lr = LinearRegression(featuresCol='feature_vector', labelCol='total_amount', maxIter=50, solver='normal')

In [17]:
lr_model = lr.fit(vtrain_data)

In [18]:
#테스트데이터도 변환
vtest_data = fitted_transform.transform(test_data)
#테스트데이터로 예측
pred = lr_model.transform(vtest_data)

In [19]:
# pred.cache()

In [20]:
pred.select('total_amount', 'prediction').show(5)

+------------+------------------+
|total_amount|        prediction|
+------------+------------------+
|       10.55|12.695522792729275|
|        13.3|14.450558014776915|
|        21.3|21.108271361254218|
|        41.3| 40.87993984204378|
|       14.15|13.906932982613997|
+------------+------------------+
only showing top 5 rows



In [21]:
lr_model.summary.rootMeanSquaredError

5.6485201652667625

In [22]:
lr_model.summary.r2

0.80849012500813